In [ ]:
%matplotlib inline

In [ ]:
%load_ext autoreload
%autoreload 1
%aimport numpy_api_stats.numpy_stats

In [ ]:
import numpy_api_stats as ns
import pandas as pd


## Results

In [ ]:
accumulated_results_fn = 'full_results_4_10_2019.csv'

In [ ]:
ns.plot_results(pd.read_csv(accumulated_results_fn),savefig=True,filename='results.png',dpi=200)

In [ ]:
accumulated_results_by_project = 'percent_by_project.csv'
det_percentages = pd.read_csv(accumulated_results_by_project).set_index('index')
f1 = ns.plot_bar_repo(det_percentages)
f1.savefig('bar_repo.png', dpi=200)

In [ ]:
det_percentages

## Analysis

In [ ]:
import os
os.environ['GOOGLE_CRED_DIR']=os.path.join(os.environ['HOME'],'creds')
os.environ['GOOGLE_APPLICATION_CREDENTIALS']=os.path.join(os.environ['GOOGLE_CRED_DIR'],'apt-footing-235018-aeb185ac9e31.json')

#### Investigate bigQuery public github dataset contents using bqhelper

In [ ]:
#from google.cloud import bigquery
github_repos = ns.BigQueryHelper(active_project= "bigquery-public-data", 
                                       dataset_name = "github_repos")

In [ ]:
github_repos.list_tables()

In [ ]:
api_results = ns.BigQueryHelper(active_project= "apt-footing-235018", 
                                       dataset_name = "NumpyAPI")

In [ ]:
api_results.list_tables()

In [ ]:
agg_results = api_results.head("latest_numpy_agg_repo_name",num_rows=10)
agg_results

### Create dataframe for plot  This pulls aggregate table from BigQuery, formats for plot, and saves to .csv 

#Create dataframe for plot  This pu
_list_agg = api_results.query_to_pandas_safe('''
SELECT * FROM `apt-footing-235018.NumpyAPI.latest_numpy_agg_repo_from_list`
''')

list_agg = _list_agg.set_index('repo_name')

list_agg = list_agg.drop(['f0_','numpy_module_numpy_count'],axis=1)

scipy_api = list_agg.loc['teoliphant/scipy']
Nscipy = scipy_api.sum()

cupy_api = list_agg.loc['cupy/cupy']
Ncupy = cupy_api.sum()

dask_api = list_agg.loc['mikegraham/dask']
Ndask = dask_api.sum()

pandas_api = list_agg.loc['pandas-dev/pandas']
Npandas = pandas_api.sum()

det_percentages = pd.DataFrame([cupy_api/Ncupy, pandas_api/Npandas, scipy_api/Nscipy, dask_api/Ndask])

det_percentages = det_percentages[ det_percentages.mean().sort_values(ascending=False).index ]

det_percentages.reset_index().to_csv('percent_by_project.csv',index=False)

### The following is some analysis of API member functions and experiments with query generation Construct API datasets

In [ ]:
import numpy as np
import pandas as pd

api = [(x, type(np.__getattribute__(x))) for x in dir(np) if not x.startswith('__')]
things = set([ x[1].__name__ for x in api])
names = set([ x[0] for x in api])
df_api = pd.DataFrame(api,columns=['name','type'])

array_type = df_api[df_api.name=='array']
things_of_array_type = df_api[df_api.type == array_type.type.values[0]]
#things_of_array_type

In [ ]:
df_api[df_api.name=='linalg']

In [ ]:
GoogleSheet = pd.read_csv('NumPy API - Sheet2.csv')
names_from_sheet = GoogleSheet['NumPy API']

In [ ]:
valid_names = set(names_from_sheet)-set(['Functions','bench',np.nan])

In [ ]:
new_names = set(names)-set(names_from_sheet)

In [ ]:
df_valid_from_sheet = df_api.set_index('name').loc[valid_names]
df_valid_from_sheet = df_valid_from_sheet.reset_index()
df_valid_from_sheet['type'] = df_valid_from_sheet.type.apply(lambda x: x.__name__)
df_valid_from_sheet.head()

#### How many of each type in the api are represented in the Google Sheet?

In [ ]:
df_valid_from_sheet[['type','name']].groupby(['type']).count().sort_values('name')

#### Of which, the following have not been implemented

In [ ]:
df_valid_from_sheet.set_index('type').loc[['CClass','NoneType','PytestTester','RClass','bool','IndexExpression','_typedict','str','nd_grid','_Feature','dict']]

#### Query Constructed for API functions..  

In [ ]:
# Selecting just a few for demonstration, Query price is independent of # of regex though, so it is better to actually run
# with all at once.

funs = ns.select(api,'function')
mods = ns.select(api,'module')
ints = ns.select(api,'int')
floats = ns.select(api,'float')
ufuncs = ns.select(api,'ufunc')
api_list = funs[:2]+mods[:2]+ints[:2]+floats[:2]+ufuncs[:2]

In [ ]:
# Use all to build full query running 
#api_list=ns.build_api_list(api)

In [ ]:
len(api_list)

In [ ]:
import numpy_api_stats as ns
apq = ns.API_QUERY_FACTORY(api_list=api_list)
#apq = ns.API_QUERY_FACTORY()
print(apq.query)

### AFter copying and pasting the above query into BigQuery and saving as sample_results... 

#####Unfortunately this doesn't work https://github.com/dask/dask/issues/3121 so we manually export results.

##### Pull data from bigQuery.  This is a one row table with a a few hundred columns so is relatively inexpensive to pull, but we'll download a local copy and work from that anyway to reduce wear on our ISP's routers.

#### references
- [Florin Badita](https://medium.com/google-cloud/naming-conventions-in-python-import-statements-a-bigquery-adventure-using-the-github-db-dump-d900159ab680)
- [Felipe Hoffa](https://medium.com/google-cloud/github-on-bigquery-analyze-all-the-code-b3576fd2b150)
- [Walker Harrison](https://dev.to/walker/using-googles-bigquery-to-better-understand-the-python-ecosystem)
- [Robert Kozikowski](https://kozikow.com/2016/06/05/more-advanced-github-code-search/)
- [Poonam Ligade](https://www.kaggle.com/poonaml/analyzing-3-million-github-repos-using-bigquery)
- [Sohier Dane](https://github.com/SohierDane/BigQuery_Helper/blob/master/bq_helper.py)
- [Google BigQuery](https://cloud.google.com/bigquery/docs/reference/legacy-sql)
- [Cloud API](https://googleapis.github.io/google-cloud-python/latest/bigquery/usage/queries.html)
- [Numpy API](https://docs.google.com/spreadsheets/d/10Tf_FK4FHaS0rpgBIuK43RHTPR601RD4p39gZQRZ2Yg/edit#gid=74265489)
- [Numpy Book](https://docs.scipy.org/doc/_static/numpybook.pdf)

#### Next steps
One possivility would be to use User defined functions [ Big Query UDF ](https://cloudplatform.googleblog.com/2015/08/Google-BigQuery-adds-UDF-support-for-deeper-cloud-analytics.html) in bigQuery for better python parsing, and thus bias reduction in api detection
Apparently there is a tool to facilitate UDF development [ UDF Test Tool](http://storage.googleapis.com/bigquery-udf-test-tool/testtool.html)

Another approach may be to search in popular repos for numpy dependencies using direct AST walking.  The following approach uses the inspection module to retrieve source code for a scipy module, and search the AST for numpy calls.  Information on AST nodes can be found at [ Green Tree Snakes ](https://greentreesnakes.readthedocs.io/en/latest/)


In [6]:
import sys
sys.path.insert(0,'/Users/mpeaton/SCIPY/build/lib.macosx-10.7-x86_64-3.7/')

In [7]:
import inspect
from scipy.linalg import basic as basic

In [ ]:
mlist= inspect.getmembers(basic)  

#### AST node walk based api search

In [3]:
import inspect
import ast
import numpy_api_stats as ns

In [9]:
tree = ast.parse(inspect.getsource(basic))

In [10]:
ns.get_npcalls(tree)

['atleast_2d',
 'atleast_1d',
 'np.concatenate',
 'atleast_1d',
 'atleast_1d',
 'np.abs',
 'np.any',
 'np.identity',
 'np.sum',
 'np.transpose',
 'np.dot',
 'atleast_2d',
 'np.ones_like',
 'np.arange',
 'np.empty_like',
 'np.arange',
 'np.asfortranarray',
 'np.iscomplexobj',
 'np.array',
 'np.zeros',
 'np.iscomplexobj',
 'np.iscomplexobj',
 'np.asarray',
 'np.asarray',
 'np.asarray',
 'np.asarray',
 'np.column_stack',
 'np.rollaxis',
 'np.rollaxis',
 'np.rollaxis',
 'np.zeros',
 'np.asarray',
 'np.conjugate',
 'np.equal',
 'np.ascontiguousarray',
 'atleast_1d',
 'np.ones_like',
 'np.iscomplexobj',
 'np.iscomplexobj',
 'np.empty',
 'np.empty',
 'np.zeros',
 'np.zeros',
 'np.finfo',
 'np.zeros',
 'np.dot',
 'np.max',
 'np.conjugate',
 'np.diag',
 'np.finfo',
 'np.common_type',
 'np.sum',
 'np.array',
 'np.finfo',
 'np.max',
 'np.finfo',
 'np.ascontiguousarray',
 'np.abs']